In [1]:
#- 패키지 임포트
import pandas as pd
from tqdm import tqdm
import re
from konlpy.tag import Okt
okt = Okt()
from collections import Counter # 형태소별 빈도 구할 때 사용
from IPython.display import clear_output
from IPython import display # 램효율을 늘리기 위해 아웃풋 display를 지워줄 것.
from ipywidgets import Output

out = Output()
display.display(out)

Output()

In [2]:
''' 파일 불러오기 '''
df = pd.read_csv("../dataset/본문_맞춤범검사/2019산림복지_본문_맞춤법검사.csv", encoding='UTF-8')
print(df.shape, df.columns)
corpus = df['correct_str'].to_list() # 문장(X문서X)으로 가져와서 실행.
# ↑ 추후에 다른 분석 시도할 때 문장 별 형태소가 필요할 때도 있음. 문장별 형태소와 이를 문서로 합친 게 있으면 편하기 때문에 문장별로 형태소 분석 실행.
print(len(corpus))

(164727, 3) Index(['split_str', 'org_idx', 'correct_str'], dtype='object')
164727


In [4]:
''' 형태소 분석 '''
nouns = []
verbs = []
adjs = []
josas = [] # 불용어 목록에 조사 사용하기 위함.
morphs = [] # 품사 태깅 없이 형태소만 추출. 추후 TF-IDF, N-gram등 텍스트 분석은 이 데이터로 진행함.
pos_list = [] # 감성분석할 때 필요.

In [5]:
for i in tqdm(range(len(corpus))):
    # 답변에서 형태소/품사 추출
    try:
        a = okt.pos(corpus[i], norm=True, stem=True) # 단어의 정규화와 어간 추출을 실행(True).
        m = okt.morphs(corpus[i], norm=True, stem=False)
        morphs.append(m) # 형태소 추가
        pos_list.append(a) # 형태소/품사 추가
        #print(i)
        for x, y in a:
            # 품사가 명사면 명사 리스트에 단어 추가
            if y == 'Noun':
                nouns.append(x)
            # 품사가 동사면 동사 리스트에 단어 추가
            elif y == 'Verb':
                verbs.append(x)
            # 품사가 형용사면 형용사 리스트에 단어 추가
            elif y == 'Adjective':
                adjs.append(x)
		    # 품사가 조사면 조사 리스트에 단어 추가.
            elif y == 'Josa':
                josas.append(x)
    except:
        pass
    #display.clear_output(wait=True) # 출력지우기

100%|██████████| 164727/164727 [1:50:03<00:00, 24.95it/s]  


In [7]:
# 명사, 동사, 형용사 리스트에서 10개 단어씩 출력해서 확인
print(nouns[0:10])
print(verbs[0:10])
print(adjs[0:10])
print(josas[0:10])

['코리아', '투데이', '뉴스', '상주시', '상주시', '상반기', '정기', '인사', '이동', '조서']
['업다', '업다', '기다', '업다', '업다', '기다', '업다', '서다', '들다', '들다']
['희다', '희다', '희다', '원하다', '밉다', '안녕하다', '이다', '있다', '좋다', '같다']
['과', '과', '과', '과', '과', '과', '과', '지', '과', '과']


In [8]:
''' 명사, 동사, 형용사, 조사 똑같이 아래 코드 반복 '''
# 명사 리스트에서 명사 빈도 리스트 생성
print(len(Counter(nouns))) # 이 값을 밑에 most_common 안에 넣으면 됨.
noun_cnt = Counter(nouns).most_common()

명사 = []
명사빈도 = []
for a, b in noun_cnt:
    명사.append(a)
    명사빈도.append(b)

# 단어와 빈도를 가지고 판다스 데이터프레임(엑셀 표와 비슷) 생성
noun_df = pd.DataFrame({'명사':명사, '빈도':명사빈도})
# 명사 데이터프레임을 파일로 저장. .xlsx, .csv, .txt 등 가능.
noun_df.to_csv("../dataset/본문_형태소분석/2019산림복지_본문_형태소분석(명사).csv", index=False, encoding='UTF-8')
noun_df

63855


,명사,빈도
0,제,44814
1,등,39322
2,수,32281
3,것,31765
4,및,30744
...,...,...
63850,한반도선진화재단,1
63851,안익태,1
63852,독제,1
63853,총본부,1


In [9]:
''' 명사, 동사, 형용사, 조사 똑같이 아래 코드 반복 '''
# 명사 리스트에서 명사 빈도 리스트 생성
print(len(Counter(verbs))) # 이 값을 밑에 most_common 안에 넣으면 됨.
verb_cnt = Counter(verbs).most_common()

동사 = []
동사빈도 = []
for a, b in verb_cnt:
    동사.append(a)
    동사빈도.append(b)

# 단어와 빈도를 가지고 판다스 데이터프레임(엑셀 표와 비슷) 생성
verb_df = pd.DataFrame({'동사':동사, '빈도':동사빈도})
# 명사 데이터프레임을 파일로 저장. .xlsx, .csv, .txt 등 가능.
verb_df.to_csv("../dataset/본문_형태소분석/2019산림복지_본문_형태소분석(동사).csv", index=False, encoding='UTF-8')
verb_df

1454


,동사,빈도
0,하다,228200
1,되다,45971
2,따르다,20272
3,보다,13235
4,되어다,11792
...,...,...
1449,출세시키다,1
1450,남아나다,1
1451,아장대다,1
1452,들러붙다,1


In [10]:
''' 명사, 동사, 형용사, 조사 똑같이 아래 코드 반복 '''
# 명사 리스트에서 명사 빈도 리스트 생성
print(len(Counter(adjs))) # 이 값을 밑에 most_common 안에 넣으면 됨.
adj_cnt = Counter(adjs).most_common()

형용사 = []
형용사빈도 = []
for a, b in adj_cnt:
    형용사.append(a)
    형용사빈도.append(b)

# 단어와 빈도를 가지고 판다스 데이터프레임(엑셀 표와 비슷) 생성
adj_df = pd.DataFrame({'형용사':형용사, '빈도':형용사빈도})
# 명사 데이터프레임을 파일로 저장. .xlsx, .csv, .txt 등 가능.
adj_df.to_csv("../dataset/본문_형태소분석/2019산림복지_본문_형태소분석(형용사).csv", index=False, encoding='UTF-8')
adj_df

1462


,형용사,빈도
0,있다,75580
1,이다,12982
2,없다,11473
3,같다,9213
4,아니다,9192
...,...,...
1457,무능력하다,1
1458,저속하다,1
1459,복스럽다,1
1460,짭짤하다,1


In [11]:
''' 명사, 동사, 형용사, 조사 똑같이 아래 코드 반복 '''
# 명사 리스트에서 명사 빈도 리스트 생성
print(len(Counter(josas))) # 이 값을 밑에 most_common 안에 넣으면 됨.
josa_cnt = Counter(josas).most_common()

조사 = []
조사빈도 = []
for a, b in josa_cnt:
    조사.append(a)
    조사빈도.append(b)

# 단어와 빈도를 가지고 판다스 데이터프레임(엑셀 표와 비슷) 생성
josa_df = pd.DataFrame({'조사':조사, '빈도':조사빈도})
# 명사 데이터프레임을 파일로 저장. .xlsx, .csv, .txt 등 가능.
josa_df.to_csv("../dataset/본문_형태소분석/2019산림복지_본문_형태소분석(조사).csv", index=False, encoding='UTF-8')
josa_df

341


,조사,빈도
0,을,166932
1,의,141246
2,에,123829
3,이,95651
4,를,80218
...,...,...
336,으로야,1
337,마따나,1
338,보다만,1
339,에게야,1


In [13]:
df["morphs"] = morphs
df

,split_str,org_idx,correct_str,morphs
0,코리아투데이뉴스 상주시 상주시 2020년 상반기 정기인사 이동조서 2020 1 1 ...,0,코리아투데이뉴스 상주시 상주시 2020년 상반기 정기인사 이동조서 2020 1 1 ...,"[코리아, 투데이, 뉴스, 상주시, 상주시, 2020년, 상반기, 정기, 인사, 이..."
1,이소영 남혜진 박소영 상하수도사업소 김인화 서원상 김주헌 윤원해 평생학습원 이세인 ...,0,이소영 남혜진 박소영 상하수도사업소 김인화 서원상 김주헌 윤 원해 평생학습원 이세인...,"[이소영, 남혜진, 박소영, 상하, 수도사, 업소, 김인, 화, 서, 원상, 김주,..."
2,안녕하세요 메리썽입니다,1,안녕하세요. 메리 성입니다,"[안녕하세요, ., 메리, 성, 입니다]"
3,혁신도시에 대해서는 많이들 들어보셨을거라고 생각이 드는데요,1,혁신도시에 대해서는 많이들 들어보셨을 거라고 생각이 드는데요,"[혁신도시, 에, 대해, 서는, 많이, 들, 들어, 보셨을, 거, 라고, 생각, 이..."
4,현재 혁신도시 시즌2가 지속적으로 진행되고 있는 만큼 지금 시기에 이에 대해 이야기...,1,현재 혁신도시 시즌2가 지속해 진행되고 있는 만큼 지금 시기에 이에 대해 이야기해보...,"[현재, 혁신도시, 시즌, 2, 가, 지속, 해, 진행, 되고, 있는, 만큼, 지금..."
...,...,...,...,...
164722,의 건립비로 지출하는 기부금 9 중소기업협동조합법 에 따른 중소기업중앙회에 중소기업...,6126,의 건립비로 지출하는 기부금 9 중소기업협동조합법에 따른 중소기업중앙회에 중소기업의...,"[의, 건립, 비, 로, 지출, 하는, 기, 부금, 9, 중소기업, 협동조합, 법,..."
164723,이하 이 호에서 같다 가 추천하는 자에게 지출하거나 대한체육회에 운동선수양성 단체경...,6126,이하 이 호에서 같다가 추천하는 자에게 지출하거나 대한체육회에 운동선수양성 단체경기...,"[이하, 이, 호, 에서, 같다가, 추천, 하는, 자, 에게, 지출, 하, 거나, ..."
164724,21 근로자직업능력 개발법 시행령 제2조에 따른 공공단체에 근로자훈련사업비로 지출하...,6126,21 근로자직업능력 개발법 시행령 제2조에 따른 공공단체에 근로자훈련사업비로 지출하...,"[21, 근로자, 직업, 능력, 개발, 법, 시행, 령, 제, 2조, 에, 따른, ..."
164725,한국국제학교 22 천진한국국제학교 23 테헤란한국학교 ㅇ 법정기부금단체 중 전문모금...,6126,한국국제학교 22 천진 한국국제학교 23 테헤란 한국학교 ㅇ 법정기부금 단체 중 전...,"[한국, 국제, 학교, 22, 천진, 한국, 국제, 학교, 23, 테헤란, 한국, ..."


In [14]:
df.to_csv("../dataset/본문_형태소분석/2019산림복지_본문_형태소분석(형태소만).csv", index=False, encoding='UTF-8')

In [15]:
df["morphs"] = pos_list
df.rename(columns={"morphs":"pos_tagging"}, inplace=True)
df

,split_str,org_idx,correct_str,pos_tagging
0,코리아투데이뉴스 상주시 상주시 2020년 상반기 정기인사 이동조서 2020 1 1 ...,0,코리아투데이뉴스 상주시 상주시 2020년 상반기 정기인사 이동조서 2020 1 1 ...,"[(코리아, Noun), (투데이, Noun), (뉴스, Noun), (상주시, N..."
1,이소영 남혜진 박소영 상하수도사업소 김인화 서원상 김주헌 윤원해 평생학습원 이세인 ...,0,이소영 남혜진 박소영 상하수도사업소 김인화 서원상 김주헌 윤 원해 평생학습원 이세인...,"[(이소영, Noun), (남혜진, Noun), (박소영, Noun), (상하, N..."
2,안녕하세요 메리썽입니다,1,안녕하세요. 메리 성입니다,"[(안녕하다, Adjective), (., Punctuation), (메리, Nou..."
3,혁신도시에 대해서는 많이들 들어보셨을거라고 생각이 드는데요,1,혁신도시에 대해서는 많이들 들어보셨을 거라고 생각이 드는데요,"[(혁신도시, Noun), (에, Josa), (대해, Noun), (서다, Ver..."
4,현재 혁신도시 시즌2가 지속적으로 진행되고 있는 만큼 지금 시기에 이에 대해 이야기...,1,현재 혁신도시 시즌2가 지속해 진행되고 있는 만큼 지금 시기에 이에 대해 이야기해보...,"[(현재, Noun), (혁신도시, Noun), (시즌, Noun), (2, Num..."
...,...,...,...,...
164722,의 건립비로 지출하는 기부금 9 중소기업협동조합법 에 따른 중소기업중앙회에 중소기업...,6126,의 건립비로 지출하는 기부금 9 중소기업협동조합법에 따른 중소기업중앙회에 중소기업의...,"[(의, Noun), (건립, Noun), (비, Noun), (로, Josa), ..."
164723,이하 이 호에서 같다 가 추천하는 자에게 지출하거나 대한체육회에 운동선수양성 단체경...,6126,이하 이 호에서 같다가 추천하는 자에게 지출하거나 대한체육회에 운동선수양성 단체경기...,"[(이하, Noun), (이, Noun), (호, Noun), (에서, Josa),..."
164724,21 근로자직업능력 개발법 시행령 제2조에 따른 공공단체에 근로자훈련사업비로 지출하...,6126,21 근로자직업능력 개발법 시행령 제2조에 따른 공공단체에 근로자훈련사업비로 지출하...,"[(21, Number), (근로자, Noun), (직업, Noun), (능력, N..."
164725,한국국제학교 22 천진한국국제학교 23 테헤란한국학교 ㅇ 법정기부금단체 중 전문모금...,6126,한국국제학교 22 천진 한국국제학교 23 테헤란 한국학교 ㅇ 법정기부금 단체 중 전...,"[(한국, Noun), (국제, Noun), (학교, Noun), (22, Numb..."


In [16]:
df.to_csv("../dataset/본문_형태소분석/2019산림복지_본문_형태소분석(품사태깅).csv", index=False, encoding='UTF-8')

In [ ]:
df.isnull().sum()